In [111]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
%matplotlib inline
!pip install gensim
import keras 
import nltk

In [112]:
from gensim.models import KeyedVectors
from google.colab import files
from nltk.corpus import stopwords

In [113]:
uploaded = files.upload()

Saving train.csv to train (2).csv
Saving train_pre_processing.csv to train_pre_processing (2).csv


ELIMINO LOS STOP WORDS

In [172]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [173]:
en_stops = set(stopwords.words('english'))

In [230]:
def remove_stopWord(texts):
  for word in texts:
    if word in en_stops:
      texts.remove(word)
  return texts

In [270]:
tw = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')), index_col=0)
tw_pp = pd.read_csv(io.StringIO(uploaded['train_pre_processing.csv'].decode('utf-8')), index_col=0)

In [271]:
tw_sinSW=tw["text"].str.lower()

In [272]:
tw_sinSW=tw_sinSW.apply(lambda x: x.split())

In [275]:
tw_sinSW.apply(lambda x:remove_stopWord(x))
tw["text_sinSW"]=tw_sinSW.apply(lambda x:remove_stopWord(x))

APLICO WORD2VECT

In [ ]:
# import modules & set up logging
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
 sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(tw["text_sinSW"], min_count=1)

In [281]:
tw["text_sinSW"][1]

['deeds', 'reason', '#earthquake', 'may', 'allah', 'forgive', 'us']

In [ ]:
model.similarity('may',"march")

In [ ]:
model.save("word2vec.model")

In [ ]:
model_work2 = Word2Vec.load("word2vec.model")

Transformo mis datos 

In [286]:
from gensim.models import KeyedVectors

In [299]:
#model_ug_sg = KeyedVectors.load('model_work2')

In [304]:
embeddings_index = {}
for w in model_work2.wv.vocab.keys():
    embeddings_index[w] = model_work2.wv[w]

In [ ]:
embeddings_index

In [291]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(tw.text_sinSW)
sequences = tokenizer.texts_to_sequences(tw.text_sinSW)

In [293]:
sequences[1]

[116, 9, 117, 686, 7414, 7415, 1905]

In [294]:
length = []
for x in tw.text_sinSW:
    length.append(len(x))
max(length)

28

In [295]:
x_train_seq = pad_sequences(sequences, maxlen=max(length))
x_train_seq[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 7413,
         650,  685,   44, 2662, 4619,   30],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  116,
           9,  117,  686, 7414, 7415, 1905],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0, 1668, 1373, 7416, 7417, 7418,
        7419,  140, 2216,  582, 1153, 1065],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 7420,
          11, 4620, 2217,  140, 1153,   39],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,   22,  991,  992,
        7421, 4621,  155, 2217, 7422,   85]], dtype=int32)

In [326]:
embedding_matrix={}
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Separo train y test

In [341]:
from sklearn.model_selection import train_test_split

In [347]:
X=tw.text
y=tw.target

In [348]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,\
                               random_state=120)

In [355]:
X_train=tokenizer.texts_to_sequences(X_train)

In [357]:
X_test=tokenizer.texts_to_sequences(X_test)

In [358]:
X_train=pad_sequences(X_train, maxlen=max(length))

In [359]:
X_test=pad_sequences(X_test, maxlen=max(length))

In [330]:
len(embedding_matrix)

27833

**APLICANDO KERAS**

In [339]:
from  keras.models  import  Sequential
from keras.layers import Embedding,Activation, Dense

In [337]:
structure_test = Sequential()
e = Embedding(len(embedding_matrix), 200, input_length=28)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.add(GlobalMaxPooling1D())
structure_test.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 200)           5566600   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 27, 100)           40100     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
Total params: 5,606,700
Trainable params: 5,606,700
Non-trainable params: 0
_________________________________________________________________


**resultados del modelo1:**

In [360]:
model_cnn_01 = Sequential()
e = Embedding(len(embedding_matrix), 200, input_length=28)
model_cnn_01.add(e)
model_cnn_01.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_01.add(GlobalMaxPooling1D())
model_cnn_01.add(Dense(256, activation='relu'))
model_cnn_01.add(Dense(1, activation='sigmoid'))
model_cnn_01.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_01.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=28, verbose=2)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6090 samples, validate on 1523 samples
Epoch 1/5
 - 20s - loss: 0.5362 - accuracy: 0.7263 - val_loss: 0.4627 - val_accuracy: 0.7859
Epoch 2/5
 - 20s - loss: 0.2592 - accuracy: 0.9064 - val_loss: 0.5002 - val_accuracy: 0.7787
Epoch 3/5
 - 21s - loss: 0.1278 - accuracy: 0.9601 - val_loss: 0.6090 - val_accuracy: 0.7820
Epoch 4/5
 - 19s - loss: 0.0870 - accuracy: 0.9737 - val_loss: 0.6083 - val_accuracy: 0.7735
Epoch 5/5
 - 19s - loss: 0.0746 - accuracy: 0.9770 - val_loss: 0.6297 - val_accuracy: 0.7715


Resultado del modelo 2

In [364]:
model_cnn_02 = Sequential()
e = Embedding(len(embedding_matrix), 200, input_length=28)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(1, activation='sigmoid'))
model_cnn_02.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_02.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=28, verbose=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6090 samples, validate on 1523 samples
Epoch 1/5
 - 20s - loss: 0.5322 - accuracy: 0.7310 - val_loss: 0.4649 - val_accuracy: 0.7905
Epoch 2/5
 - 20s - loss: 0.2592 - accuracy: 0.8997 - val_loss: 0.5023 - val_accuracy: 0.7873
Epoch 3/5
 - 19s - loss: 0.1234 - accuracy: 0.9644 - val_loss: 0.6134 - val_accuracy: 0.7807
Epoch 4/5
 - 19s - loss: 0.0949 - accuracy: 0.9709 - val_loss: 0.5926 - val_accuracy: 0.7735
Epoch 5/5
 - 21s - loss: 0.0724 - accuracy: 0.9772 - val_loss: 0.6626 - val_accuracy: 0.7682


In [ ]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [362]:
model_cnn_03 = Sequential()
e = Embedding(len(embedding_matrix), 200, input_length=28)
model_cnn_03.add(e)
model_cnn_03.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_03.add(GlobalMaxPooling1D())
model_cnn_03.add(Dense(256, activation='relu'))
model_cnn_03.add(Dense(1, activation='sigmoid'))
model_cnn_03.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_03.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32, verbose=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6090 samples, validate on 1523 samples
Epoch 1/5
 - 18s - loss: 0.5398 - accuracy: 0.7286 - val_loss: 0.4659 - val_accuracy: 0.7899
Epoch 2/5
 - 19s - loss: 0.2653 - accuracy: 0.8970 - val_loss: 0.5063 - val_accuracy: 0.7873
Epoch 3/5
 - 18s - loss: 0.1223 - accuracy: 0.9599 - val_loss: 0.5721 - val_accuracy: 0.7859
Epoch 4/5
 - 17s - loss: 0.0920 - accuracy: 0.9716 - val_loss: 0.6231 - val_accuracy: 0.7571
Epoch 5/5
 - 17s - loss: 0.0792 - accuracy: 0.9757 - val_loss: 0.6192 - val_accuracy: 0.7774
